# Chapter 33: Advanced Helm

While Chapter 32 established foundational Helm concepts for packaging Kubernetes applications, this chapter explores sophisticated patterns required for enterprise-scale deployments. Advanced Helm usage encompasses reusable library charts for organizational standardization, complex templating logic for dynamic configuration, and secure secret management integration.

Understanding subchart architectures, chart testing methodologies, and the Open Container Initiative (OCI) registry support enables teams to manage hundreds of microservices across multiple environments while maintaining security compliance and deployment reliability.

## 33.1 Subchart Development

Complex applications decompose into logical components (frontend, backend, database) managed as separate charts but deployed as a unified release through subcharts.

### Subchart Architecture

Subcharts reside in the `charts/` directory, installed automatically with the parent:

```text
enterprise-platform/
├── Chart.yaml
├── values.yaml
├── charts/
│   ├── frontend-1.2.0.tgz      # Packaged dependency
│   ├── backend-2.0.0.tgz       # Another dependency
│   └── database/               # Unpacked subchart (development)
│       ├── Chart.yaml
│       ├── values.yaml
│       └── templates/
└── templates/
```

### Parent-Child Value Passing

Values flow hierarchically from parent to child through scoped namespaces:

```yaml
# Parent values.yaml
global:
  environment: production
  domain: company.com
  imageRegistry: registry.company.com

frontend:
  replicaCount: 3
  image:
    tag: v1.2.0
  ingress:
    enabled: true
    host: app.{{ .Values.global.domain }}  # Templated in child

backend:
  replicaCount: 5
  database:
    host: localhost  # Referenced by child
    
database:
  enabled: true
  postgresql:
    auth:
      username: platform_user
      database: platform_db
```

**Accessing in child templates:**
```yaml
# charts/frontend/templates/deployment.yaml
replicas: {{ .Values.replicaCount }}
env:
  - name: API_ENDPOINT
    value: http://{{ .Release.Name }}-backend:8080
  - name: ENVIRONMENT
    value: {{ .Values.global.environment }}
```

### Conditional Subchart Activation

Enable or disable components based on environment or requirements:

```yaml
# Chart.yaml (Parent)
dependencies:
  - name: postgresql
    version: 12.x.x
    repository: https://charts.bitnami.com/bitnami
    condition: database.enabled  # Toggles installation
    alias: database
  
  - name: redis
    version: 17.x.x
    repository: https://charts.bitnami.com/bitnami
    condition: redis.enabled
  
  - name: monitoring
    version: 1.x.x
    repository: https://charts.company.com
    condition: monitoring.enabled
```

**Environment-specific activation:**
```yaml
# values-production.yaml
database:
  enabled: true
  postgresql:
    replication:
      enabled: true

redis:
  enabled: true
  cluster:
    enabled: true

monitoring:
  enabled: true

# values-development.yaml  
database:
  enabled: true  # But single instance
  
redis:
  enabled: false  # Use in-memory cache

monitoring:
  enabled: false  # Dev doesn't need full monitoring
```

### Sharing Templates Between Subcharts

Parent charts can define templates used by children:

```yaml
# Parent templates/_labels.tpl
{{- define "enterprise-platform.standardLabels" -}}
helm.sh/chart: {{ .Chart.Name }}-{{ .Chart.Version }}
app.kubernetes.io/managed-by: {{ .Release.Service }}
environment: {{ .Values.global.environment }}
tenant: {{ .Values.global.tenant | default "shared" }}
{{- end }}
```

```yaml
# Child chart using parent's template
metadata:
  labels:
    {{- include "enterprise-platform.standardLabels" . | nindent 4 }}
```

**Note:** Child charts can access parent templates, but not vice versa. Use library charts (Section 33.2) for shared logic usable by both.

## 33.2 Library Charts

Library charts (Chart type: `library`) contain reusable template definitions without deployable resources, functioning as template "include files" for application charts.

### Creating Library Charts

```yaml
# common/Chart.yaml
apiVersion: v2
name: common
description: Common template helpers for company charts
type: library  # Critical distinction
version: 1.5.0
```

```yaml
# common/templates/_resources.tpl
{{/* Define standard resource constraints */}}
{{- define "common.resources.standard" -}}
requests:
  memory: "256Mi"
  cpu: "250m"
limits:
  memory: "512Mi"
  cpu: "500m"
{{- end }}

{{/* Define high-performance resources */}}
{{- define "common.resources.high" -}}
requests:
  memory: "2Gi"
  cpu: "1000m"
limits:
  memory: "4Gi"
  cpu: "2000m"
{{- end }}

{{/* Standard security context */}}
{{- define "common.securityContext" -}}
runAsNonRoot: true
runAsUser: 1000
seccompProfile:
  type: RuntimeDefault
capabilities:
  drop:
    - ALL
{{- end }}

{{/* Common labels */}}
{{- define "common.labels.standard" -}}
helm.sh/chart: {{ .Chart.Name }}-{{ .Chart.Version | replace "+" "_" }}
{{ include "common.selectorLabels" . }}
app.kubernetes.io/managed-by: {{ .Release.Service }}
{{- end }}

{{- define "common.selectorLabels" -}}
app.kubernetes.io/name: {{ include "common.name" . }}
app.kubernetes.io/instance: {{ .Release.Name }}
{{- end }}

{{- define "common.name" -}}
{{- default .Chart.Name .Values.nameOverride | trunc 63 | trimSuffix "-" }}
{{- end }}
```

### Consuming Library Charts

```yaml
# Application Chart.yaml
apiVersion: v2
name: payment-service
type: application
version: 2.1.0
dependencies:
  - name: common
    version: 1.x.x
    repository: https://charts.company.com/library
```

```yaml
# Application templates/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: {{ include "common.name" . }}
  labels:
    {{- include "common.labels.standard" . | nindent 4 }}
spec:
  template:
    spec:
      containers:
        - name: app
          resources:
            {{- include "common.resources.standard" . | nindent 12 }}
          securityContext:
            {{- include "common.securityContext" . | nindent 12 }}
```

### Library Chart Best Practices

1. **Version Pinning:** Use semantic version ranges (`1.x.x`) to receive patches but not breaking changes
2. **Documentation:** Library charts require extensive documentation of available templates and parameters
3. **Testing:** Validate library charts with a "test application" chart that consumes all templates
4. **No Values:** Library charts should not define `values.yaml` (ignored during installation), only template defaults

## 33.3 Advanced Templating Techniques

Beyond basic variable substitution, Helm templates support sophisticated logic for complex configurations.

### Pipeline Syntax

Pipes (`|`) chain functions left-to-right, similar to Unix pipes:

```yaml
# Chain multiple transformations
name: {{ .Values.name | upper | quote | nindent 4 }}

# Default with fallback chain
image: {{ .Values.image | default .Values.global.image | default "nginx:latest" | quote }}

# Complex formatting
labels: {{ .Values.labels | toYaml | trim | nindent 8 }}
```

### Control Flow Patterns

**Looping with Index:**
```yaml
# Access index in range
env:
{{- range $index, $val := .Values.items }}
  - name: ITEM_{{ $index }}
    value: {{ $val | quote }}
{{- end }}
```

**Conditional Blocks:**
```yaml
# Complex condition with and/or
{{- if and .Values.ingress.enabled (or .Values.tls.enabled .Values.global.tls.enabled) }}
tls:
  - hosts:
      - {{ .Values.ingress.host | quote }}
    secretName: {{ .Values.ingress.tlsSecret }}
{{- end }}
```

**With Blocks for Scope:**
```yaml
# Change scope to avoid repetition
{{- with .Values.securityContext }}
securityContext:
  runAsUser: {{ .runAsUser }}
  runAsGroup: {{ .runAsGroup }}
  fsGroup: {{ .fsGroup }}
{{- end }}
```

### String Manipulation

```yaml
# Regex replacement (Sprig functions)
sanitizedName: {{ .Values.name | regexReplaceAll "[^a-zA-Z0-9]+" "-" | lower | trunc 63 }}

# Split and join
parts: {{ .Values.domain | split "." | first }}
fullPath: {{ .Values.paths | join "," }}

# Indentation helpers
config: |
  {{ .Values.config | indent 4 }}
```

### Debugging Templates

```bash
# Render templates without installation
helm template myapp ./chart --values values-production.yaml --debug

# Show computed values
helm get values myapp --all

# Lint with strict validation
helm lint ./chart --strict

# Validate against Kubernetes API (dry-run)
helm install myapp ./chart --dry-run=server --values values.yaml
```

**Template Debugging Tips:**
```yaml
# Add diagnostic annotations
metadata:
  annotations:
    helm.sh/template-debug: |
      Values: {{ .Values | toJson | nindent 6 }}
      Capabilities: {{ .Capabilities | toJson | nindent 6 }}
```

## 33.4 Helmfile

Helmfile (third-party tool) manages multiple Helm releases declaratively, enabling environment-specific deployments and release ordering.

### Helmfile Structure

```yaml
# helmfile.yaml
releases:
  # Infrastructure layer
  - name: ingress-nginx
    namespace: ingress
    chart: ingress-nginx/ingress-nginx
    version: 4.7.0
    values:
      - values/ingress-nginx.yaml.gotmpl  # Templated values
    hooks:
      - events: ["prepare"]
        showlogs: true
        command: "kubectl"
        args: ["create", "namespace", "ingress", "--dry-run=client", "-o", "yaml", "|", "kubectl", "apply", "-f", "-"]

  # Data layer
  - name: postgresql
    namespace: database
    chart: bitnami/postgresql
    version: 12.1.0
    installed: {{ .Values | get "postgresql.enabled" true }}
    values:
      - values/postgresql.yaml
    secrets:
      - secrets/postgresql.yaml  # SOPS-encrypted

  # Application layer
  - name: payment-service
    namespace: payments
    chart: ./charts/payment-service  # Local chart
    version: 2.1.0
    needs:
      - database/postgresql  # Wait for dependency
    values:
      - values/payment-service/common.yaml
      - values/payment-service/{{ .Environment.Name }}.yaml
    set:
      - name: image.tag
        value: {{ requiredEnv "IMAGE_TAG" }}
      - name: global.environment
        value: {{ .Environment.Name }}

  - name: notification-service
    namespace: notifications
    chart: ./charts/notification-service
    needs:
      - payments/payment-service
    values:
      - values/notification-service.yaml

# Environments
environments:
  development:
    values:
      - environments/dev.yaml
  staging:
    values:
      - environments/staging.yaml
  production:
    values:
      - environments/production.yaml

# Repositories
repositories:
  - name: ingress-nginx
    url: https://kubernetes.github.io/ingress-nginx
  - name: bitnami
    url: https://charts.bitnami.com/bitnami

# Hooks
hooks:
  - events: ["presync"]
    command: "echo"
    args: ["Starting deployment to {{ .Environment.Name }}"]
```

### Templated Values (.gotmpl)

Helmfile processes `.gotmpl` files through Go templates before passing to Helm:

```yaml
# values/postgresql.yaml.gotmpl
auth:
  username: {{ .Values | get "postgresql.username" "postgres" }}
  password: {{ requiredEnv "DB_PASSWORD" }}
  
primary:
  persistence:
    enabled: true
    storageClass: {{ .Values | get "storage.class" "standard" }}
    size: {{ .Values | get "postgresql.storage.size" "10Gi" }}
    
resources:
  {{- if eq .Environment.Name "production" }}
  requests:
    memory: "4Gi"
    cpu: "2000m"
  {{- else }}
  requests:
    memory: "1Gi"
    cpu: "500m"
  {{- end }}
```

### Helmfile Commands

```bash
# Apply all releases
helmfile apply

# Apply specific environment
helmfile -e production apply

# Diff changes before applying
helmfile diff

# Sync (install/upgrade) specific release
helmfile -l name=payment-service sync

# Destroy all releases
helmfile destroy

# Template all releases (useful for CI validation)
helmfile template
```

## 33.5 Chart Testing

The `helm test` framework validates chart functionality by running test pods that verify deployment correctness.

### Test Structure

```yaml
# templates/tests/test-connection.yaml
apiVersion: v1
kind: Pod
metadata:
  name: "{{ include "myapp.fullname" . }}-test-connection"
  labels:
    {{- include "myapp.labels" . | nindent 4 }}
  annotations:
    "helm.sh/hook": test
    "helm.sh/hook-delete-policy": hook-succeeded
spec:
  containers:
    - name: wget
      image: busybox:1.36
      command: ['wget']
      args: 
        - '--spider'
        - '--timeout=5'
        - '{{ include "myapp.fullname" . }}:{{ .Values.service.port }}/health'
  restartPolicy: Never
```

```yaml
# templates/tests/test-integration.yaml
apiVersion: v1
kind: Pod
metadata:
  name: "{{ include "myapp.fullname" . }}-test-integration"
  annotations:
    "helm.sh/hook": test-success  # Marks success on completion
spec:
  containers:
    - name: test-runner
      image: "{{ .Values.image.repository }}:{{ .Values.image.tag }}"
      command: ['npm', 'run', 'test:integration']
      env:
        - name: API_ENDPOINT
          value: http://{{ include "myapp.fullname" . }}
        - name: TEST_DB_HOST
          value: {{ .Release.Name }}-postgresql
  restartPolicy: Never
```

### Running Tests

```bash
# Run tests after installation
helm install myapp ./myapp --wait
helm test myapp

# Run with logs
helm test myapp --logs

# Run specific test
helm test myapp --filter name=myapp-test-integration

# Cleanup failed tests
helm test myapp --cleanup
```

### CI/CD Integration

```yaml
# GitLab CI
test:chart:
  stage: test
  image: alpine/helm:latest
  script:
    # Lint
    - helm lint ./chart
    
    # Unit test (if using helm-unittest plugin)
    - helm unittest ./chart
    
    # Install to temporary namespace
    - helm install test-release ./chart 
        --namespace test-${CI_JOB_ID} 
        --create-namespace
        --values values-ci.yaml
        --wait
        --timeout 5m
    
    # Run Helm tests
    - helm test test-release --namespace test-${CI_JOB_ID}
    
    # Cleanup
    - helm uninstall test-release --namespace test-${CI_JOB_ID}
    - kubectl delete namespace test-${CI_JOB_ID}
```

## 33.6 Security Hardening

### Chart Signing and Provenance

Establish trust through cryptographic signatures:

```bash
# Generate PGP key (one-time)
gpg --full-generate-key

# Export public key for distribution
gpg --armor --export "Platform Team" > public.key

# Sign chart
helm package --sign --key "Platform Team" --keyring ~/.gnupg/secring.gpg ./myapp

# Verify signature
helm verify myapp-1.0.0.tgz

# Install with verification
helm install --verify myapp ./myapp-1.0.0.tgz
```

**Provenance File Structure:**
```yaml
# myapp-1.0.0.tgz.prov
-----BEGIN PGP SIGNED MESSAGE-----
Hash: SHA512

apiVersion: v2
appVersion: 1.16.0
description: A Helm chart for Kubernetes
name: myapp
type: application
version: 1.0.0

...
files:
  myapp-1.0.0.tgz: sha256:abc123...
-----BEGIN PGP SIGNATURE-----
...
-----END PGP SIGNATURE-----
```

### OCI Registry Support (Helm 3.8+)

Store charts in container registries alongside images:

```bash
# Enable experimental OCI support (Helm 3.7-3.7, default in 3.8+)
export HELM_EXPERIMENTAL_OCI=1

# Login to registry
helm registry login registry.company.com -u username -p password

# Save chart to local OCI layout
helm chart save ./myapp registry.company.com/charts/myapp:1.0.0

# Push to registry
helm chart push registry.company.com/charts/myapp:1.0.0

# Pull and install
helm chart pull registry.company.com/charts/myapp:1.0.0
helm chart export registry.company.com/charts/myapp:1.0.0
helm install myapp ./myapp
```

**OCI advantages:**
- Unified storage with container images
- Existing registry authentication/authorization
- Content-addressable storage

### RBAC and Service Accounts

```yaml
# templates/rbac.yaml
{{- if .Values.rbac.create }}
apiVersion: v1
kind: ServiceAccount
metadata:
  name: {{ include "myapp.serviceAccountName" . }}
  annotations:
    {{- with .Values.serviceAccount.annotations }}
    {{- toYaml . | nindent 4 }}
    {{- end }}
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: {{ include "myapp.fullname" . }}
rules:
  - apiGroups: [""]
    resources: ["configmaps"]
    verbs: ["get", "list", "watch"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: {{ include "myapp.fullname" . }}
subjects:
  - kind: ServiceAccount
    name: {{ include "myapp.serviceAccountName" . }}
    namespace: {{ .Release.Namespace }}
roleRef:
  kind: Role
  name: {{ include "myapp.fullname" . }}
  apiGroup: rbac.authorization.k8s.io
{{- end }}
```

## 33.7 External Secret Management

### Sealed Secrets (Bitnami)

Encrypt secrets for safe storage in Git:

```bash
# Install kubeseal CLI
brew install kubeseal

# Create sealed secret from existing secret
kubectl create secret generic my-secret \
  --from-literal=password=supersecret \
  --dry-run=client -o yaml | \
  kubeseal --controller-namespace=sealed-secrets \
           --controller-name=sealed-secrets \
           --format yaml > sealed-secret.yaml

# Commit sealed-secret.yaml to Git
# Apply to cluster
kubectl apply -f sealed-secret.yaml
```

**Helm integration:**
```yaml
# templates/sealed-secret.yaml
{{- if .Values.sealedSecrets.enabled }}
apiVersion: bitnami.com/v1alpha1
kind: SealedSecret
metadata:
  name: {{ include "myapp.fullname" . }}-credentials
  annotations:
    sealedsecrets.bitnami.com/cluster-wide: "false"
spec:
  encryptedData:
    password: {{ .Values.sealedSecrets.password }}
{{- end }}
```

### External Secrets Operator

Synchronize secrets from cloud providers:

```yaml
# templates/external-secret.yaml
{{- if .Values.externalSecrets.enabled }}
apiVersion: external-secrets.io/v1beta1
kind: ExternalSecret
metadata:
  name: {{ include "myapp.fullname" . }}-db-credentials
spec:
  refreshInterval: "1h"
  secretStoreRef:
    name: aws-secrets-manager
    kind: ClusterSecretStore
  target:
    name: {{ include "myapp.fullname" . }}-db-credentials
    creationPolicy: Owner
  data:
    - secretKey: password
      remoteRef:
        key: production/myapp/database
        property: password
    - secretKey: username
      remoteRef:
        key: production/myapp/database
        property: username
{{- end }}
```

### Helm Secrets Plugin (SOPS)

Mozilla SOPS integration for encrypted values files:

```bash
# Install plugin
helm plugin install https://github.com/jkroepke/helm-secrets --version v4.5.0

# Encrypt values file
sops -e -i secrets.yaml

# Use in Helm
helm secrets install myapp ./chart -f values.yaml -f secrets.yaml
```

**SOPS configuration (.sops.yaml):**
```yaml
creation_rules:
  - path_regex: secrets/.*\.yaml$
    kms: arn:aws:kms:us-east-1:123456789:key/12345678-1234-1234-1234-123456789012
    pgp: 'Platform Team Key ID'
```

## 33.8 Optimization and Best Practices

### Chart Size Optimization

**Exclude unnecessary files:**
```text
# .helmignore
*.md
.git/
.gitignore
*.tgz
ci/
test-values/
```

**Compress large ConfigMaps:**
```yaml
# Compress configuration if > 1MB
data:
  config.yaml: |
    {{ .Files.Get "config/large.yaml" | b64enc }}
```

### Upgrade Strategies

**Atomic upgrades:**
```bash
helm upgrade --install myapp ./chart --atomic --timeout 5m
```

**Force resource updates:**
```bash
helm upgrade myapp ./chart --force --cleanup-on-fail
```

**Rollback on failure:**
```bash
helm upgrade myapp ./chart --wait --wait-for-jobs
# Automatically rolls back if timeout exceeded
```

### Troubleshooting

**Failed release cleanup:**
```bash
# List failed releases
helm list --all --failed

# Rollback
helm rollback myapp 2

# Force uninstall stuck release
helm uninstall myapp --no-hooks --keep-history
```

**Debugging template rendering:**
```bash
# Show computed values
helm get values myapp --all

# Show manifest diff
helm diff upgrade myapp ./chart

# Template with debug
helm template myapp ./chart --debug 2>&1 | less
```

---

## Chapter Summary and Preview

In this chapter, we explored advanced Helm patterns essential for enterprise Kubernetes operations. Subchart architectures enable decomposition of complex applications into manageable, independently versioned components while maintaining unified deployment lifecycle. Library charts provide organizational standardization through reusable template definitions, ensuring consistent security contexts, resource constraints, and labeling schemes across hundreds of microservices. Advanced templating techniques utilizing pipeline syntax and Sprig functions enable dynamic configuration generation, while Helmfile orchestrates multi-release deployments across environments with dependency ordering and secret integration. Chart testing through the `helm test` framework validates functionality post-deployment, ensuring that templates render correctly and applications respond to health checks. Security hardening encompasses chart signing with PGP for provenance verification, OCI registry support for unified artifact storage, and RBAC templating for least-privilege service accounts. External secret management integration—via Sealed Secrets, External Secrets Operator, or SOPS—eliminates plaintext credential storage in version control, maintaining compliance requirements while preserving GitOps workflows.

**Key Takeaways:**
- Use library charts (type: library) to centralize organizational standards for security contexts, resource limits, and common labels, versioning them independently from application charts
- Implement `helm test` hooks for post-deployment validation, including connectivity tests and integration verification, running these in CI/CD before promoting releases between environments
- Store charts in OCI-compliant container registries (Helm 3.8+) alongside Docker images to leverage existing authentication mechanisms and artifact lifecycle policies
- Encrypt sensitive values using SOPS with the Helm Secrets plugin or use External Secrets Operator to synchronize secrets from cloud provider secret managers, never committing plaintext credentials to Git
- Use Helmfile for multi-service deployments to manage release ordering (needs), environment-specific value composition, and secrets decryption in a declarative manner

**Next Chapter Preview:**
Chapter 34: Kubernetes Admission Controllers examines advanced security and policy enforcement mechanisms in Kubernetes. We will explore ValidatingAdmissionWebhooks and MutatingAdmissionWebhooks for custom policy enforcement, Open Policy Agent (OPA) Gatekeeper for declarative policy as code, and Kyverno for Kubernetes-native policy management. This chapter addresses how to enforce organizational standards—such as requiring specific labels, blocking privileged containers, or mandating resource limits—at the API server level, preventing non-compliant resources from entering the cluster before Helm charts are even deployed.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='32. helm_kubernetes_package_manager.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='34. kubernetes_admission_controllers.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
